<a href="https://colab.research.google.com/github/suwatoh/Python-learning/blob/main/201_Beautiful_Soup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Beautiful Soup
==============

Web スクレイピング
------------------

**スクレイピング**（scraping）とは、「こすり落とす」という意味を持つ scrape に由来する用語で、Web やデータベースを広く探って特定の情報を抽出する手法を指す。

Web API が提供されていないサイトからデータを取得するために、Web ページのスクレイピングが行われる。ただし、以下の事項に注意する。

  * **Web サイトが利用規約においてスクレイピングによるデータ取得を禁止している場合**
  * **ダウンロードの間隔を1秒以上空けていないなどの理由で、Web サイトのサーバーに過重な負担をかける場合**
  * **取得したデータの使用が、個人や家族間での活用、情報解析、Web 検索サービスの活用といった著作権法上許容されている限度を超える場合**（情報を頒布・販売する行為など）
  * **Web サイト自体がコンテンツを不法に複製・頒布している場合**

これらの場合には、スクレイピング行為が違法とされ法的責任を問われることになる。

なお、Web 検索サービスの活用を目的とする場合は、サイトの `robots.txt` ファイルで特定のユーザーエージェントでのアクセスが許可されていない URL パスを対象としたスクレイピングをしないように気を付けること。

Web スクレイピングは、大まかに 3 つのステップに分けることができる。

  1. データの取得
  2. データの抽出
  3. データの保存

Requests パッケージは、データの取得に使用される。

一方、[Beautiful Soup](https://pypi.org/project/beautifulsoup4/) は HTML や XML を解析する機能を提供するサードパティ製パッケージであり、Web スクレイピングにおけるデータの抽出に使用される。ライセンスは MIT License。 インストール方法は次のとおり。

``` python
pip install beautifulsoup4
```

オブジェクトの種類
------------------

### BeautifulSoup オブジェクト ###

`bs4.BeautifulSoup` オブジェクトは、与えられたマークアップが解析されて作られた木構造を表現する。コンストラクタのオプションは次のとおり。

``` python
bs4.BeautifulSoup(markup="", features=None, builder=None, parse_only=None, from_encoding=None, exclude_encodings=None, element_classes=None, **kwargs)
```

主に使用するのは、次の 3 つのオプションである。

| オプション | 意味 |
|:---|:---|
| `markup` | パースするマークアップを文字列、バイト列またはファイルオブジェクトで指定する。Requests を使って取得したデータをパースする場合は、その `text` 属性よりも `content` <br />属性を渡したほうが文字化けしない |
| `features` | 使用するパーサーを指定する。`None` を指定した場合（デフォルト）、標準ライブラリの `'html.parser'` モジュールが使用される |
| `from_encoding` | マークアップの文字コードを文字列で指定する。`None` を指定した場合（デフォルト）、文字コードが自動で検出される。自動検出の結果が間違っている場合は、文字化けとな<br />り、正しいパースが行われない |
| `exclude_encodings` | 除外する文字コードを文字列のリストで指定する。間違った文字コードが自動検出される場合は、それを含むリストを渡すと、正しい文字コードが推測されることがある |

`features` オプションに指定できるパーサーは次のとおり。

| パーサー | マークアップ | 特徴 |
|:---|:---|:---|
| `'html.parser'` | HTML | Python 標準ライブラリの `html.parser` モジュールが提供するパーサー |
| `'html5lib'` | HTML | [html5lib](https://pypi.org/project/html5lib/) パッケージが提供するパーサー。標準規格に従っていない HTML でもパースしてくれる。低速 |
| `'lxml'` | HTML | [lxml](https://pypi.org/project/lxml/) パッケージが提供するパーサー。C で書かれているため高速だが、標準規格に従っていない HTML の解析に失敗する |
| `'lxml-xml'` または `'xml'` | XML | `lxml` パッケージが提供するパーサー。XML をパースする |

基本的に `'lxml'` を指定し、うまくいかない場合は `'html.parser'` を使うようにすればよい。ただし、lxml パッケージをインストールする必要がある。

マークアップを文字列で指定する例:

In [ ]:
from bs4 import BeautifulSoup
print(BeautifulSoup("<html><head></head><body>Sacr&eacute; bleu!</body></html>"))

<html><head></head><body>Sacré bleu!</body></html>


マークアップをファイルオブジェクトで指定する例:

``` python
from bs4 import BeautifulSoup

with open("index.html") as fp:
    soup = BeautifulSoup(fp, 'lxml')
```

### タグオブジェクト ###

`bs4.BeautifulSoup` オブジェクトを作成すると、与えられたマークアップが解析され、`bs4.Tag` オブジェクト（以下、タグオブジェクト）と `bs4.NavigableString` オブジェクトを節点とするツリーが構築される。`bs4.Tag` と `bs4.NavigableString` はともに `bs4.PageElement` クラスのサブクラスである。

タグオブジェクトは、マークアップの要素を表す。`bs4.BeautifulSoup` は `bs4.Tag` のサブクラスであり、ツリーのルートを表す。

親要素のタグオブジェクトから、ドット `.` 記法と要素名で子要素のタグオブジェクトにアクセスできる（直接の子要素でなくてもよい）。ただしこの方法では、マークアップ中に同じタグが複数含まれていても、親要素のタグから見て最初に現れるものに対応するタグオブジェクトにしかアクセスできないことに注意する。

In [ ]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<html><head><title>My Title</title></head><body><p class="sentence">Some sentence</p></body></html>')
soup.title  # title 要素にアクセス

<title>My Title</title>

### タグオブジェクトの属性 ###

タグオブジェクトの主な属性は次のとおり。

| 属性 | 意味 |
|:---|:---|
| `name` | タグ名の文字列 |
| `attrs` | タグの属性の名前と値の組からなる辞書。複数の値を持つことが許される属性（`class` など）の値はリストになっている |
| `contents` | （読み出し専用）直接の子オブジェクトのリスト |
| `children` | （読み出し専用）直接の子オブジェクトを yield するジェネレーター |
| `descendants` | （読み出し専用）全ての子孫オブジェクトを再帰的に yield するジェネレーター |
| `string` | （読み出し専用）このタグに `bs4.NavigableString` である子が 1 つある場合、戻り値はその `bs4.NavigableString` オブジェクト。この要素に子タグが 1 つある場合、戻り値は<br />その子の `Tag.string`（再帰的）。このタグに子がないか、複数の子がある場合、戻り値は `None` |
| `strings` | （読み出し専用）子孫要素の `bs4.NavigableString` オブジェクトを yield するジェネレーター |

タグオブジェクトに対し添字表記（`[key]`）を使うと、タグの属性を参照できる。キー `key` が存在しない場合はエラーが発生する。

In [21]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<html><head><title>My Title</title></head><body><p class="sentence">Some sentence</p></body></html>')
assert soup.title.name == 'title'
assert soup.p.attrs == {'class': ['sentence']}
assert soup.p['class'] == ['sentence']
for i, elem in enumerate(soup.descendants):
    print(i, elem)

0 <html><head><title>My Title</title></head><body><p class="sentence">Some sentence</p></body></html>
1 <head><title>My Title</title></head>
2 <title>My Title</title>
3 My Title
4 <body><p class="sentence">Some sentence</p></body>
5 <p class="sentence">Some sentence</p>
6 Some sentence


タグオブジェクトの属性は、一部の読み出し専用プロパティを除いて、変更可能である。この変更は、直ちにツリー全体に反映される。

In [14]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<html><head><title>My Title</title></head><body><p class="sentence">Some sentence</p></body></html>')
soup.title.string = 'New Title'
soup.p['id'] = 'verybold'
print(soup)

<html><head><title>New Title</title></head><body><p class="sentence" id="verybold">Some sentence</p></body></html>


### タグオブジェクトのメソッド ###

``` python
tag.prettify(encoding=None, formatter="minimal")
```

このメソッドは、ツリーをタグ付きのテキストとして書式化して返す。

In [ ]:
from bs4 import BeautifulSoup

html_doc = '''<html><head><title>テスト HTML</title></head>
<body>
<div>
<p class="overview">テスト<b>TEST</b>てすと
'''
soup = BeautifulSoup(html_doc, "html.parser")
print(soup.prettify())

<html>
 <head>
  <title>
   テスト HTML
  </title>
 </head>
 <body>
  <div>
   <p class="overview">
    テスト
    <b>
     TEST
    </b>
    てすと
   </p>
  </div>
 </body>
</html>



``` python
tag.get(key, default=None)
```

このメソッドは、名前が `key` の属性の値を返す。そのような属性が存在しない場合は、`default` を返す。`tag.attrs.get(key, default)` と等価である。

In [ ]:
from bs4 import BeautifulSoup

html_doc = '<div><p class="overview">テスト</p></div>'
soup = BeautifulSoup(html_doc, "html.parser")
assert soup.p.get("class") ==  ["overview"]

``` python
tag.get_attribute_list(key, default=None)
```

このメソッドは、`get()` メソッドと同様であるが、唯一の違いが、複数値の属性であってもそうでなくても、常にリストとして値を取得することである。

In [ ]:
from bs4 import BeautifulSoup

id_soup = BeautifulSoup('<p id="my id"></p>', 'html.parser')
id_soup.p.get_attribute_list('id')

['my id']

``` python
tag.find(name=None, attrs={}, recursive=True, string=None, **kwargs)
```

このメソッドは、タグオブジェクトを起点に、ツリーを下に向かって探索し、引数に指定した検索条件にマッチした最初のタグオブジェクトか `bs4.NavigableString` オブジェクトを返す。`tag.p` は `tag.find("p")` と等価である。`find()` が何も見つけられなかった場合、 `None` を返す。

| 引数 | 意味 |
|:---|:---|
| `name` | 要素の名前（タグ名）で検索する。以下を指定できる<br><br>・文字列: その文字列とタグ名が完全に一致するものを検索する（例: `soup.find('b')`）<br><br>・正規表現オブジェクト: その正規表現を使用して `search()` メソッドを使ってフィルタリングを行う（例: `soup.find(re.compile("^b"))`）<br><br>・文字列のリスト: リスト内の いずれか のアイテムに対して文字列マッチを行う（例: `soup.find(['a', 'b'])`）<br><br>・`True`: 可能な限り全てのタグにマッチする（例: `soup.find(True)`）<br><br>・関数: 要素を唯一の引数として受け取り、引数が条件にマッチする場合は `True` を、そうでない場合は `False` を返す関数を指定できる（例: `soup.find(myfunc)`） |
| `attrs` | 辞書で指定した要素の属性名と値で検索する。単独の文字列を与えることもでき、`{"class": 文字列}` と同等 |
| `recursive` | `False` の場合、直下の子要素のみが検索対象となる |
| `string` | タグで囲まれたテキストを指定する。これを指定すると、戻り値の型は `bs4.NavigableString` オブジェクトとなる。指定の方法は `name` 引数と同じ |
| `kwargs` | キーワード引数として属性の名前と値を与えてフィルターを指定する。ただし、`class` は Python の予約語なのでアンダースコアを付けて `class_` とする。指定の方法は `name` <br />引数と同じ |

複数の引数で検索条件を指定した場合は、それらの条件の AND 検索が行われる。

In [21]:
from bs4 import BeautifulSoup, NavigableString
import re

html_doc = """<div>
<p><b>属性なし</ b></p>
<p class="overview noise">class 属性あり</p>
<p class="overview" id="123">class 属性と id 属性あり</p>
<p><a href="https://www.python.org/">python.org トップページ</a></p>
<p><a href="https://docs.python.org/ja/3/">Python 公式ドキュメント</a></p>
</div>
"""
soup = BeautifulSoup(html_doc, "html.parser")
assert soup.find("p").text == "属性なし"  # soup.p.text と等価
assert soup.find(re.compile("^b")).name == 'b'
assert soup.find(attrs={"class": "overview"}).text == "class 属性あり"  # class 属性は値の 1 つと一致すればよい
assert soup.find(attrs={"class": "overview noise"}).text == "class 属性あり"  # class 属性は複数の値の並びが一致することをテストすることもできる
assert soup.find("p", "overview", id="123").text == soup.find("p", class_="overview", id="123").text == "class 属性と id 属性あり"
assert soup.find(href=re.compile(r"/3/$")).text == "Python 公式ドキュメント"
t = soup.find(string="python.org トップページ")
assert isinstance(t, NavigableString)  # 戻り値の型は bs4.NavigableString
assert t == "python.org トップページ"

def has_class_but_no_id(tag):
    return tag.has_attr('class') and not tag.has_attr('id')

soup.find_all(has_class_but_no_id)

[<p class="overview noise">class 属性あり</p>]

``` python
tag.find_all(name=None, attrs={}, recursive=True, string=None, limit=None, **kwargs)
```

このメソッドは、タグオブジェクトを起点に、ツリーを下に向かって探索し、引数に指定した検索条件にマッチしたタグオブジェクトや `bs4.NavigableString` オブジェクトのリスト `bs4.ResultSet` オブジェクトを返す（`bs4.ResultSet` は `list` のサブクラスである）。`find_all()` が何も見つけられなかった場合、空のリストを返す。

引数の `limit` に指定した数だけ要素が見つかった時点で探索を終了する。`limit` が `None` の場合（デフォルト）、すべての要素を探索する。その他の引数の意味は、`find()` メソッドと同様である。

タグオブジェクトは呼び出し可能オブジェクトでもあり、`()` を付けて呼び出すことができる。`tag(*args, **kwargs)` は `tag.find_all(*args, **kwargs)` と等価である。

In [22]:
from bs4 import BeautifulSoup, NavigableString
import re

html_doc = """<div>
<p><b>属性なし</ b></p>
<p class="overview noise">class 属性あり</p>
<p class="overview" id="123">class 属性と id 属性あり</p>
<p><a href="https://www.python.org/">python.org トップページ</a></p>
<p><a href="https://docs.python.org/ja/3/">Python 公式ドキュメント</a></p>
</div>
"""
soup = BeautifulSoup(html_doc, "html.parser")
for elem in soup.find_all(["a", "b"]):
    print(elem)
print("=" * 70)
for elem in soup.find_all(True):
    print(elem)
    print("-" * 70)

<b>属性なし</b>
<a href="https://www.python.org/">python.org トップページ</a>
<a href="https://docs.python.org/ja/3/">Python 公式ドキュメント</a>
<div>
<p><b>属性なし</b></p>
<p class="overview noise">class 属性あり</p>
<p class="overview" id="123">class 属性と id 属性あり</p>
<p><a href="https://www.python.org/">python.org トップページ</a></p>
<p><a href="https://docs.python.org/ja/3/">Python 公式ドキュメント</a></p>
</div>
----------------------------------------------------------------------
<p><b>属性なし</b></p>
----------------------------------------------------------------------
<b>属性なし</b>
----------------------------------------------------------------------
<p class="overview noise">class 属性あり</p>
----------------------------------------------------------------------
<p class="overview" id="123">class 属性と id 属性あり</p>
----------------------------------------------------------------------
<p><a href="https://www.python.org/">python.org トップページ</a></p>
----------------------------------------------------------------------
<a hre

``` python
tag.select(selector, namespaces=None, limit=None, **kwargs)
```

このメソッドは、CSS セレクターを使って、マッチするタグオブジェクトのリスト `bs4.ResultSet` オブジェクトを返す。

| 引数 | 意味 |
|:---|:---|
| `selector` | CSS セレクターを文字列で指定する |
| `namespaces` | XML の名前空間を辞書で指定する |
| `limit` | 指定した数だけ要素が見つかった時点で探索を終了する |

CSS セレクターの実装は、Soup Sieve パッケージによって行われている（`pip` を使用して Beautiful Soup をインストールした場合、Soup Sieve も同時にインストールされている）。Soup Sieve のドキュメントには、現在サポートされている[全ての CSS セレクターのリスト](https://facelessuser.github.io/soupsieve/selectors/)がある。

In [3]:
from bs4 import BeautifulSoup

html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

soup = BeautifulSoup(html_doc, 'html.parser')
print(f'{soup.select("p > a:nth-of-type(2)")=}')

soup.select("p > a:nth-of-type(2)")=[<a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>]


``` python
tag.select_one(selector, namespaces=None, **kwargs)
```

このメソッドは、CSS セレクターを使って、マッチする最初のタグオブジェクトを返す。引数の意味は `select()` メソッドと同じ。

このほか、タグオブジェクトは、`css` 属性（読み取り専用プロパティ）を通じて Soup Sieve API をサポートしている。実のところ、`tag.select()` メソッドと `tag.select_one()` メソッドも、`tag.css.select()` と `tag.css.select_one()` のショートカットである。`css` プロパティがサポートする他の Soup Sieve API は以下の通り。

| Soup Sieve API | 機能 |
|:---|:---|
| `tag.css.iselect(select, namespaces=None, limit=0, **kwargs)` | `select()` と同様に動作する、リストではなくジェネレーターを返す |
| `tag.css.closest(select, namespaces=None, **kwargs)` | タグオブジェクトの先祖の中で、指定した CSS セレクターに一致する最も近いオブジェクトを返す |
| `tag.css.match(select, namespaces=None, **kwargs)` | タグオブジェクトが CSS セレクターに一致するかどうかに基づいてブール値を返す |
| `tag.css.filter(select, namespaces=None, **kwargs: Any)` | 指定した CSS セレクターに一致するタグの直接の子要素の部分集合を返す |

In [18]:
from bs4 import BeautifulSoup

html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

soup = BeautifulSoup(html_doc, 'html.parser')
assert [tag['id'] for tag in soup.css.iselect(".sister")] == ['link1', 'link2', 'link3']
elsie = soup.css.select_one(".sister")
assert elsie.css.closest("p.story").name == 'p'
assert elsie.css.match("#link1")
assert [tag.string for tag in soup.find('p', 'story').css.filter('a')] == ['Elsie', 'Lacie', 'Tillie']

### NavigableString オブジェクト ###

`bs4.NavigableString` は、タグで囲まれたテキストを表す。タグに囲まれたテキストもツリーの節点となっていて、`bs4.NavigableString` オブジェクトがこれに対応している。**タグの周りの空白（改行を含む）も含まれる**ことに注意する。

`bs4.NavigableString` は、`str` のサブクラスでもあり、`==` 演算子や `!=` 演算子で文字列と比較することができる。

`bs4.NavigableString` オブジェクトを Unicode 文字列に変換するには、`str` を使用する。

タグオブジェクトの `string` 属性は、唯一の直接の子である `bs4.NavigableString` オブジェクトを得る。つまり、`string` 属性は、タグオブジェクトから延びる枝が `bs4.NavigableString` オブジェクトだけである場合に、それを参照する。タグオブジェクトの子要素が 1 つだけ存在する場合は、その子要素の `string` 属性を参照する。この参照は再帰的に行われる。タグが複数の要素を含んでいる場合、`string` 属性が何を指すべきかが不明確になるため、`string` 属性は `None` を返す。

タグの中に複数の要素がある場合でも、`bs4.NavigableString` オブジェクトを取得するには、タグオブジェクトの `strings` 属性が与えるジェネレーターを使用する。

In [27]:
from bs4 import BeautifulSoup, NavigableString

html_doc = '''<body>
<div>
<p class="overview">テスト <b>TEST</b>てすと</p>
</div>
</body>
'''
soup = BeautifulSoup(html_doc, "html.parser")
print(f"{soup.string=}")
print(f"{soup.div.string=}")
print(f"{soup.p.string=}")
print(f"{soup.b.string=}")
assert isinstance(soup.b.string, NavigableString)  # string 属性が返すのは文字列ではない
for t in soup.strings:
    assert isinstance(t, NavigableString)
[x for x in soup.strings]

soup.string=None
soup.div.string=None
soup.p.string=None
soup.b.string='TEST'


['\n', '\n', 'テスト ', 'TEST', 'てすと', '\n', '\n', '\n']

### 特別な文字列 ###

特別な文字列を表すため、以下の `bs4.NavigableString` サブクラスが定義されている。

| サブクラス | 意味 | プレフィックス | サフィックス |
|:---|:---|:---|:---|
| `bs4.element.PreformattedString` | 基底クラス。プレフィックスとサフィックスを持つ | `''` | `''` |
| `bs4.element.CData` ★ | CDATA ブロックを表すクラス | `'<![CDATA['` | `']]>'` |
| `bs4.element.ProcessingInstruction` ★ | SGML を表すクラス。SGML は XML の前身になったフォーマット | `'<?'` | `'>'` |
| `bs4.element.XMLProcessingInstruction` | XML を表すクラス | `'<?'` | `'?>'` |
| `bs4.element.Comment` ★ | HTML と XML のコメントを表すクラス | `'<!--'` | `'-->'` |
| `bs4.element.Declaration` ★ | XML の宣言を表すクラス | `'<?'` | `'?>'` |
| `bs4.element.Doctype` ★ | DOCTYPE を表すクラス | `'<!DOCTYPE '` | `'>\n'` |
| `bs4.element.Stylesheet` ★ | スタイルシートを表すクラス | | |
| `bs4.element.Script` ★ | スクリプトを表すクラス。JavaScript など | | |
| `bs4.element.TemplateString` ★ | 大きな文書の中に埋め込まれた HTML テンプレートの中で見られる文字列を表すクラス | | |
| `bs4.element.RubyTextString` | ルビ文字を指定する `<rt>` タグのテキストを表すクラス | | |
| `bs4.element.RubyParenthesisString` | ルビ代替表示文字を指定する `<rp>` タグのテキストを表すクラス | | |

★印は、名前空間 `bs4` でもインポートされている。例えば、`bs4.element.Comment` は `bs4.Comment` で参照できる。

### クラス階層 ###

Beautiful Soup の主要なクラスの階層は次のようになっている。

``` text
PageElement
 ├───── Tag
 │            └── BeautifulSoup
 │
 │
 └──┐
       ├── NavigableString
str  ─┘      ├─ PreformattedString
               │    ├─ CData
               │    ├─ ProcessingInstruction
               │    ├─ XMLProcessingInstruction
               │    ├─ Comment
               │    ├─ Declaration
               │    ├─ Doctype
               ├─ Stylesheet
               ├─ Script
               ├─ TemplateString
               ├─ RubyTextString
               └─ RubyParenthesisString
```

以降で取り上げる属性とメソッドは、ほとんどが `bs4.PageElement` の属性とメソッドを継承したものとなる。したがって、`bs4.NavigableString` オブジェクトでも使うことができる。

テキストの取得
--------------

要素内のタグで囲まれた文字列（テキスト）の取得に関する属性は、以下の通り。

| 引数 | 意味 |
|:---|:---|
| `text` | （読み出し専用）要素のテキスト（文字列）。`get_text()` と同じ |
| `stripped_strings` | （読み出し専用）子孫要素のテキスト（文字列）を返すジェネレーター。余分な空白を取り除いたものを返す |

`text` 属性は、`get_text()` メソッドを引数なしで呼び出すプロパティである。`get_text()` メソッドを直接呼び出す場合、以下の引数を指定できる。

``` python
get_text(separator="", strip=False, types=object())
```

| 引数 | 意味 |
|:---|:---|
| `separator` | タグで区切られていた位置に指定した文字列を挿入する |
| `strip` | `True` を指定するとタグの周りの空白（改行を含む）を取り除く |
| `types` | `bs4.NavigableString` のサブクラスをタプルで指定できる。標準では取得できない要素を取得する際に指定する |

`get_text()` メソッドは、デフォルトでは `bs4.NavigableString` と `bs4.CData` オブジェクトを再帰的に取得し、文字列化して返す。つまり、`get_text()` と `get_text(types=(bs4.NavigableString, bs4.CData))` は等価である。

`text` 属性と `get_text()` メソッドは、テキストを再帰的に取得し、連結する。子要素のテキストも含まれた文字列を取得できる。`strip=True` を指定しない限り改行も含まれることに注意する。

In [ ]:
from bs4 import BeautifulSoup

html_doc = """<div>
  red
  <p>b l u e</p>
  <span> green </span>
</div>
"""

soup = BeautifulSoup(html_doc, "html.parser")
print(soup.get_text())  # soup.text と同じ
print("-" * 30)
print(soup.get_text(strip=True))
print("-" * 30)
print(soup.get_text(strip=True, separator="++"))


  red
  b l u e
 green 


------------------------------
redb l u egreen
------------------------------
red++b l u e++green


In [ ]:
from bs4 import BeautifulSoup, Script

html_doc = '<div><script>alert("Hello world!!");</script></div>'
soup = BeautifulSoup(html_doc, "html.parser")
print(soup.get_text())  # デフォルトでは <script> タグの中身は取得できない
print("-" * 30)
print(soup.get_text(types=(Script,)))


------------------------------
alert("Hello world!!");


`text` 属性ではタグの周りの空白（改行を含む）が含まれてしまうが、`stripped_strings` 属性を使用すると、それらを削除した文字列を取得することができる。`stripped_strings` 属性はジェネレーターであることに注意する。

In [29]:
from bs4 import BeautifulSoup, NavigableString

html_doc = '''<body>
<div>
<p class="overview">テスト <b>TEST</b>てすと</p>
</div>
</body>
'''
soup = BeautifulSoup(html_doc, "html.parser")
print(f"{soup.text=}")
print(f"{soup.div.text=}")
print(f"{soup.p.text=}")
print(f"{soup.b.text=}")
[x for x in soup.stripped_strings]

soup.text='\n\nテスト TESTてすと\n\n\n'
soup.div.text='\nテスト TESTてすと\n'
soup.p.text='テスト TESTてすと'
soup.b.text='TEST'


['テスト', 'TEST', 'てすと']

要素の取得（移動）
------------------

### 上への移動 ###

上への移動に関する属性は、以下の通り。

| 属性 | 意味 |
|:---|:---|
| `parent` | 親要素 |
| `parents` | 全ての祖先要素を再帰的に yield するジェネレーター |

In [42]:
from bs4 import BeautifulSoup
soup = BeautifulSoup('<html><head><title>My Title</title></head><body><p class="sentence">Some sentence</p></body></html>')
assert soup.title.parent.name == 'head'
assert soup.title.string.parent.name == 'title'
assert isinstance(soup.html.parent, BeautifulSoup)  # <html> のようなトップレベルのタグの親は BeautifulSoup オブジェクト自体
assert soup.parent is None
for parent in soup.title.string.parents:
    print(parent.name)  # BeautifulSoup.name は [document] となることに注意

title
head
html
[document]


上への移動に関するメソッドは、以下の通り。

``` python
find_parent(name=None, attrs={}, **kwargs)
find_parents(name=None, attrs={}, limit=None, **kwargs)
```

これらのメソッドは、オブジェクトを起点に、ツリーを上に向かって探索し、引数に指定した検索条件にマッチしたタグオブジェクトか bs4.NavigableString オブジェクを返す。`find_parents()` メソッドは全てのオブジェクトのリストである `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

これらの検索メソッドは、`parents` を使ってすべての親を繰り返しチェックし、提供されたフィルターに一致するかどうかを確認している。

### 横への移動 ###

横への移動に関する属性は、以下の通り。

| 属性 | 意味 |
|:---|:---|
| `next_sibling` | 同じ階層にある兄弟オブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト）で文書の 1 つ下に現れるもの |
| `next_siblings` | 同じ階層にある兄弟オブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト）で文書の下に現れるもの全てを yield するジェネレーター |
| `previous_sibling` | 同じ階層にある兄弟オブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト）で文書の 1 つ上に現れるもの |
| `previous_siblings` | 同じ階層にある兄弟オブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト）で文書の上に現れるもの全てを yield するジェネレーター |

In [52]:
from bs4 import BeautifulSoup
soup = BeautifulSoup("<a><b>text1</b><c>text2</c>text3</a>", 'html.parser')
print(f"{soup.b.next_sibling=}")
print(f"{soup.c.previous_sibling=}")
print(f"{soup.b.previous_sibling=}")
print(f"{soup.c.next_sibling=}")
print("-" * 40)
for sibling in soup.b.next_siblings:
    print(repr(sibling))

soup.b.next_sibling=<c>text2</c>
soup.c.previous_sibling=<b>text1</b>
soup.b.previous_sibling=None
soup.c.next_sibling='text3'
----------------------------------------
<c>text2</c>
'text3'


横への移動に関するメソッドは、以下の通り。

``` python
find_next_sibling(name=None, attrs={}, string=None, **kwargs)
find_next_siblings(name=None, attrs={}, string=None, limit=None, **kwargs)
```

これらのメソッドは、`next_siblings` を使用して、兄弟オブジェクト（タグオブジェクトか bs4.NavigableString オブジェクト）を順にたどり、引数に指定した検索条件にマッチしたものを返す。`find_next_siblings()` メソッドはマッチした全てのオブジェクトのリストである `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

``` python
find_previous_sibling(name=None, attrs={}, string=None, **kwargs)
find_previous_siblings(name=None, attrs={}, string=None, limit=None, **kwargs)
```

これらのメソッドは、`previous_siblings` を使用して、ツリー内でオブジェクトより前にある兄弟オブジェクト（タグオブジェクトか bs4.NavigableString オブジェクト）を順にたどり、引数に指定した検索条件にマッチしたものを返す。`find_previous_siblings()` メソッドはマッチした全てのオブジェクトのリストである `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

### 前後の移動 ###

前後の移動に関する属性は、以下の通り。

| 属性 | 意味 |
|:---|:---|
| `next_element` | このオブジェクトの直後に解析されたオブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト） |
| `next_elements` | このオブジェクトの直後以降に解析されたオブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト）を yield するジェネレーター |
| `previous_element` | このオブジェクトの直前に解析されたオブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト） |
| `previous_elements` | このオブジェクトの直前までに解析されたオブジェクト（タグオブジェクトか `bs4.NavigableString` オブジェクト）を yield するジェネレーター |

In [10]:
html_doc = """
<html><head><title>The Dormouse's story</title></head>
<body>
<p class="title"><b>The Dormouse's story</b></p>

<p class="story">Once upon a time there were three little sisters; and their names were
<a href="http://example.com/elsie" class="sister" id="link1">Elsie</a>,
<a href="http://example.com/lacie" class="sister" id="link2">Lacie</a> and
<a href="http://example.com/tillie" class="sister" id="link3">Tillie</a>;
and they lived at the bottom of a well.</p>

<p class="story">...</p>
"""

from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')
first_a_tag = soup.a
assert first_a_tag["id"] == "link1"
print(f"{first_a_tag.next_element=}")
print(f"{first_a_tag.previous_element=}")
for i, elem in enumerate(first_a_tag.next_elements):
    print(i, elem)

first_a_tag.next_element='Elsie'
first_a_tag.previous_element='Once upon a time there were three little sisters; and their names were\n'
0 Elsie
1 ,

2 <a class="sister" href="http://example.com/lacie" id="link2">Lacie</a>
3 Lacie
4  and

5 <a class="sister" href="http://example.com/tillie" id="link3">Tillie</a>
6 Tillie
7 ;
and they lived at the bottom of a well.
8 

9 <p class="story">...</p>
10 ...
11 



前後の移動に関するメソッドは、以下の通り。

``` python
find_next(name=None, attrs={}, string=None, **kwargs)
find_all_next(name=None, attrs={}, string=None, limit=None, **kwargs)
```

これらのメソッドは、`next_elements` を使用して、文書内でその要素の後に出現するタグや文字列を順にたどり、引数に指定した検索条件にマッチしたものを返す。`find_all_next()` メソッドはマッチした全てのオブジェクトのリストである `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

``` python
find_previous(name=None, attrs={}, string=None, **kwargs)
find_all_previous(name=None, attrs={}, string=None, limit=None, **kwargs)
```

これらのメソッドは、`previous_elements` を使用して、文書内でその要素の前に出現したタグや文字列を順にたどり、引数に指定した検索条件にマッチしたものを返す。`find_all_previous()` メソッドはマッチした全てのオブジェクトのリストである `bs4.ResultSet` オブジェクトを返す。引数の意味は、`find()` および `find_all()` と同様である。

ツリーの変更
------------

``` python
soup.new_tag(name, namespace=None, nsprefix=None, attrs={}, sourceline=None, sourcepos=None, **kwattrs)
```

これは、`bs4.BeautifulSoup` オブジェクトのメソッドで、新しいタグを作成し、タグオブジェクトを返す。主な引数は次のとおり。

| 引数 | 意味 |
|:---|:---|
| `name` | 新しいタグの名前 |
| `attrs` | 新しいタグの属性の名前と値を辞書で指定する |
| `kwattrs` | 新しいタグの属性の名前と値をキーワード引数で指定する |

``` python
soup.new_string(s, subclass=None):
```

これは、`bs4.BeautifulSoup` オブジェクトのメソッドで、新しいテキストを作成し、`bs4.NavigableString` オブジェクトを返す。`subclass` に `bs4.NavigableString` のサブクラスを指定できる。

``` python
soup.wrap(wrap_inside)
```

このメソッドは、このオブジェクトの親要素として `wrap_inside` に指定したタグオブジェクトをツリーに挿入する。

``` python
soup.unwrap()
```

このメソッドは、このオブジェクトだけを取り除いて、子要素は残す。

``` python
elem.extract()
```

これは `PageElement` のメソッドで、このオブジェクトを削除し、親要素を返す。

``` python
elem.decompose()
```

これは `PageElement` のメソッドで、自身を削除する。子孫オブジェクトもすべて削除される。

``` python
tag.clear(decompose=False)
```

これはタグオブジェクトのメソッドで、子要素をすべて削除する。`decompose` 引数を `True` に指定すると、子要素の削除に `decompose()` メソッドが使われる。

なお、要素の属性を削除するためには、del 文を使って `del tag["name"]` のように書く。



In [ ]:
from bs4 import BeautifulSoup

soup = BeautifulSoup("<p></p>")
new_div = soup.new_tag("div")
print(soup.p.wrap(new_div))
print(soup.p.unwrap())

<div><p></p></div>
<p></p>
